In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import json
pd.options.display.max_rows = 999

In [2]:
managers_url = 'https://football.fantasysports.yahoo.com/f1/528229/teams'
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = True)
browser.visit(managers_url)
managers_html = browser.html
managers_soup = BeautifulSoup(managers_html, "html.parser")
managers_table = managers_soup.findAll("tbody")[1]
managers = managers_table.findAll("tr")
team_names = []
for m in managers:
    manager = m.findAll('td')
    team_name = manager[0].text
    team_names.append(team_name)
manager_names = ["Matt", "Ron", "Jake", "Doug", "Chi Shing", "Dai", 
                 "Evan", "Rajiv", "G-Way", "Sean", "Ryan", "Andrew"]
manager_list = []
for x in np.arange(len(team_names)):
    manager_list.append({"team_name": team_names[x], "manager": manager_names[x]})
# print(manager_list)
manager_df = pd.DataFrame(manager_list)
manager_df.head(12)

,manager,team_name
0,Matt,2 Gurley's 1 Cup
1,Ron,Bilbro Naggins
2,Jake,Brodo Naggins
3,Doug,Bye Week
4,Chi Shing,Chi ShingT's Team
5,Dai,crushiNG RON
6,Evan,Cry me a Philip
7,Rajiv,FirstRoundFlops
8,G-Way,G
9,Sean,Kamara Sutra


In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = True)
draft_url = "https://football.fantasysports.yahoo.com/f1/528229/draftresults"
browser.visit(draft_url)
draft_html = browser.html
draft_soup = BeautifulSoup(draft_html, "html.parser")
draft_table = draft_soup.findAll("tbody")[1]
draft_picks = draft_table.findAll("tr")
draft_pick_list = []
for picks in draft_picks:
    pick = picks.findAll("td")
    name = pick[1].text.split(" (")[0]
    team = pick[1].text.split(" (")[1].split(" - ")[0]
    pos = pick[1].text.split(" (")[1].split(" - ")[1].split(")")[0]
    draft_pick_dict = {
        "name": name,
        "NFLTeam": team,
        "Pos": pos,
        "cost": int(pick[2].text[1:]),
        "team": pick[3].text
    }
    draft_pick_list.append(draft_pick_dict)
for player in draft_pick_list:
    if player['name'] == "Todd Gurley II":
        player["cost"] = 40
    if player["name"] == "Dalvin Cook":
        player["cost"] = 64

In [4]:
draft_df = pd.DataFrame(draft_pick_list)
draft_df['keeper'] = ""
draft_df["manager"] = ""
draft_df.head(10)

,NFLTeam,Pos,cost,name,team,keeper,manager
0,Dal,QB,6,Dak Prescott,Brodo Naggins,,
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,,
2,NE,TE,64,Rob Gronkowski,crushiNG RON,,
3,Jax,RB,60,Leonard Fournette,Kamara Sutra,,
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,,
5,NYG,RB,74,Saquon Barkley,Bye Week,,
6,Buf,RB,29,LeSean McCoy,Nags,,
7,LAC,QB,11,Philip Rivers,Thx Ron and Le'Veon,,
8,Hou,RB,28,Lamar Miller,Cry me a Philip,,
9,Cin,WR,67,A.J. Green,Brodo Naggins,,


In [5]:
keeper_df = pd.read_csv('keeper.csv')
keeper_df.head(15)

,player,duration
0,Dalvin Cook,B
1,Todd Gurley II,AB
2,Adam Thielen,B
3,DeAndre Hopkins,B
4,Melvin Gordon III,AB
5,Davante Adams,AB
6,Kenyan Drake,B
7,Golden Tate,AB
8,Corey Clement,B
9,Devonta Freeman,AB


In [6]:
for i, d in draft_df.iterrows():
    for index, keeper in keeper_df.iterrows():
        if d['name'] == keeper['player']:
            draft_df.at[i,'keeper'] = keeper['duration']
            
draft_df.head(10)

,NFLTeam,Pos,cost,name,team,keeper,manager
0,Dal,QB,6,Dak Prescott,Brodo Naggins,,
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,,
2,NE,TE,64,Rob Gronkowski,crushiNG RON,,
3,Jax,RB,60,Leonard Fournette,Kamara Sutra,,
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,,
5,NYG,RB,74,Saquon Barkley,Bye Week,,
6,Buf,RB,29,LeSean McCoy,Nags,,
7,LAC,QB,11,Philip Rivers,Thx Ron and Le'Veon,,
8,Hou,RB,28,Lamar Miller,Cry me a Philip,,
9,Cin,WR,67,A.J. Green,Brodo Naggins,,


In [17]:
for i, d in draft_df.iterrows():
    for index, team in manager_df.iterrows():
        if d["team"] == team["team_name"]:
            draft_df.at[i, "manager"] = team["manager"]
draft_df.head(15)

,NFLTeam,Pos,cost,name,team,keeper,manager
0,Dal,QB,6,Dak Prescott,Brodo Naggins,,Jake
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,,Chi Shing
2,NE,TE,64,Rob Gronkowski,crushiNG RON,,Dai
3,Jax,RB,60,Leonard Fournette,Kamara Sutra,,Sean
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,,Evan
5,NYG,RB,74,Saquon Barkley,Bye Week,,Doug
6,Buf,RB,29,LeSean McCoy,Nags,,Ryan
7,LAC,QB,11,Philip Rivers,Thx Ron and Le'Veon,,Andrew
8,Hou,RB,28,Lamar Miller,Cry me a Philip,,Evan
9,Cin,WR,67,A.J. Green,Brodo Naggins,,Jake


In [8]:
transaction_pages = np.arange(5)*25
transaction_list = []
for page in transaction_pages:
    try:
        transaction_url = 'https://football.fantasysports.yahoo.com/f1/528229/transactions?transactionsfilter=all&count=' + str(page)
        executable_path = {'executable_path': 'chromedriver.exe'}
        browser = Browser('chrome', **executable_path, headless = True)
        browser.visit(transaction_url)
        transaction_html = browser.html
        transaction_soup = BeautifulSoup(transaction_html, "html.parser")
        transaction_table = transaction_soup.findAll("tbody")[1]
        transactions = transaction_table.findAll("tr")
        for t in transactions:
            transaction = t.findAll('td')
            transaction_info = transaction[1].text.splitlines()
            team_info = transaction[2].text.splitlines()
#             print(transaction_info)
#             print(team_info)
            try:
                try:
                    transaction_info.remove("Q")
                except:
                    pass
                try:
                    transaction_info.remove("D")
                except:
                    pass
                try:
                    transaction_info.remove("O")
                except:
                    pass
                try:
                    transaction_info.remove("NA")
                except:
                    pass
                if transaction_info[3].strip()[0] == "$":
                    price = int(transaction_info[3].strip().split("  ")[0][1:])
                else:
                    price = int(0)
                tm = team_info[3].strip()
                for i, t in manager_df.iterrows():
                    if t['team_name'] == tm:
                        transaction_dict = {
                            "add": transaction_info[1].strip(),
                            "nfl_team": transaction_info[2].strip().split(" - ")[0],
                            "position": transaction_info[2].strip().split(" - ")[1],
                            "price":price,
                            "drop":transaction_info[4].strip(),
                            "ffteam": t['manager'],
                            "type": "waiver"
                        }
                        transaction_list.append(transaction_dict)
            except:
                pass
    except:
        pass



In [9]:
trade_list = []
trade00a = {"index": 53.3,
           "add": "Marquise Goodwin",
           "nfl_team": "SF",
           "position": "WR",
           "price":5,
           "drop":"Adrian Peterson",
           "ffteam":"Matt",
           "type": "trade"}
trade00b = {"index": 53.6,
           "add": "Adrian Peterson",
           "nfl_team": "WAS",
           "position": "RB",
           "price":12,
           "drop":"Marquise Goodwin",
           "ffteam":"Jake",
           "type": "trade"}
trade_list.append(trade00a)
trade_list.append(trade00b)
MGIndex = draft_df.index[draft_df['name']==trade_list[0]['add']]
APIndex = draft_df.index[draft_df['name']==trade_list[1]['add']]

In [10]:
transaction_df = pd.DataFrame(transaction_list)
transaction_df = transaction_df.reindex(index=transaction_df.index[::-1]).reset_index(drop=True)
transaction_df = transaction_df.drop([48, 49]).reset_index(drop=True)
transaction_df.head(10)

,add,drop,ffteam,nfl_team,position,price,type
0,James White,Pittsburgh,Jake,NE,RB,2,waiver
1,Mason Crosby,Baker Mayfield,Matt,GB,K,0,waiver
2,John Ross,Dez Bryant,Dai,Cin,WR,0,waiver
3,Austin Hooper,Charles Clay,Dai,Atl,TE,0,waiver
4,Graham Gano,Donte Moncrief,Sean,Car,K,0,waiver
5,Allen Hurns,LeGarrette Blount,Ron,Dal,WR,0,waiver
6,Quincy Enunwa,Denver,Ron,NYJ,WR,0,waiver
7,Jaylen Samuels,Austin Hooper,Dai,Pit,"RB,TE",0,waiver
8,Geronimo Allison,Dede Westbrook,Ron,GB,WR,0,waiver
9,Dede Westbrook,Matt Prater,G-Way,Jax,WR,0,waiver


In [11]:
trade_df = pd.DataFrame(trade_list)
trade_df.set_index('index', inplace=True)
trade_df.head()
transaction_df = pd.concat([transaction_df, trade_df])

In [12]:
transaction_df = transaction_df.sort_index()
transaction_df.head(10)

,add,drop,ffteam,nfl_team,position,price,type
0.0,James White,Pittsburgh,Jake,NE,RB,2,waiver
1.0,Mason Crosby,Baker Mayfield,Matt,GB,K,0,waiver
2.0,John Ross,Dez Bryant,Dai,Cin,WR,0,waiver
3.0,Austin Hooper,Charles Clay,Dai,Atl,TE,0,waiver
4.0,Graham Gano,Donte Moncrief,Sean,Car,K,0,waiver
5.0,Allen Hurns,LeGarrette Blount,Ron,Dal,WR,0,waiver
6.0,Quincy Enunwa,Denver,Ron,NYJ,WR,0,waiver
7.0,Jaylen Samuels,Austin Hooper,Dai,Pit,"RB,TE",0,waiver
8.0,Geronimo Allison,Dede Westbrook,Ron,GB,WR,0,waiver
9.0,Dede Westbrook,Matt Prater,G-Way,Jax,WR,0,waiver


In [13]:
for x, t in transaction_df.iterrows():
    for i, d in draft_df.iterrows():
        if t['type'] == "waiver":
            if d['name'] == t['drop']:
                draft_df.at[i, 'name'] = t['add']
                draft_df.at[i, 'NFLTeam'] = t['nfl_team']
                draft_df.at[i, 'Pos'] = t['position']
                draft_df.at[i, 'cost'] = t['price']
                draft_df.at[i, 'keeper'] = ""
        if t["type"] == "trade":
            if i == MGIndex:
                draft_df.at[i, "manager"] = "Matt"
            if i == APIndex:
                draft_df.at[i, "manager"] = "Jake"   
roster_df = draft_df
roster_df.head(15)

,NFLTeam,Pos,cost,name,team,keeper,manager
0,Dal,QB,6,Dak Prescott,Brodo Naggins,,Jake
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,,Chi Shing
2,NE,TE,64,Rob Gronkowski,crushiNG RON,,Dai
3,Jax,RB,60,Leonard Fournette,Kamara Sutra,,Sean
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,,Evan
5,NYG,RB,74,Saquon Barkley,Bye Week,,Doug
6,Buf,RB,29,LeSean McCoy,Nags,,Ryan
7,LAC,QB,11,Philip Rivers,Thx Ron and Le'Veon,,Andrew
8,Hou,RB,28,Lamar Miller,Cry me a Philip,,Evan
9,Cin,WR,67,A.J. Green,Brodo Naggins,,Jake


In [14]:
for i, r in roster_df.iterrows():
    for index, t in manager_df.iterrows():
        if r['manager'] == t['manager']:
            roster_df.at[i, "team"] = t["team_name"]
            
roster_df.head(15)

,NFLTeam,Pos,cost,name,team,keeper,manager
0,Dal,QB,6,Dak Prescott,Brodo Naggins,,Jake
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,,Chi Shing
2,NE,TE,64,Rob Gronkowski,crushiNG RON,,Dai
3,Jax,RB,60,Leonard Fournette,Kamara Sutra,,Sean
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,,Evan
5,NYG,RB,74,Saquon Barkley,Bye Week,,Doug
6,Buf,RB,29,LeSean McCoy,Nags,,Ryan
7,LAC,QB,11,Philip Rivers,Thx Ron and Le'Veon,,Andrew
8,Hou,RB,28,Lamar Miller,Cry me a Philip,,Evan
9,Cin,WR,67,A.J. Green,Brodo Naggins,,Jake


In [15]:
ff_team_names = roster_df.team.unique()
print(ff_team_names)

['Brodo Naggins' "Chi ShingT's Team" 'crushiNG RON' 'Kamara Sutra'
 'Cry me a Philip' 'Bye Week' 'Nags' "Thx Ron and Le'Veon"
 "2 Gurley's 1 Cup" 'FirstRoundFlops' 'Bilbro Naggins' 'G']


In [19]:
roster_df = roster_df[['name', 'NFLTeam', 'Pos', 'cost', 'keeper', 'team']].sort_values(by=["cost"], ascending=False)
for ff in ff_team_names:
    print(roster_df.loc[roster_df['team'] == ff])
    print("------------------------------------------------------------------------------------")

                  name NFLTeam  Pos  cost keeper           team
9           A.J. Green     Cin   WR    67         Brodo Naggins
180    Ezekiel Elliott     Dal   RB    55      B  Brodo Naggins
28           Zach Ertz     Phi   TE    49         Brodo Naggins
23       Brandin Cooks     LAR   WR    39         Brodo Naggins
178       Kirk Cousins     Min   QB    22     AB  Brodo Naggins
177     Michael Thomas      NO   WR    18     AB  Brodo Naggins
80       Rashaad Penny     Sea   RB    13         Brodo Naggins
35     Adrian Peterson     Was   RB    12         Brodo Naggins
58         Los Angeles     LAR  DEF    11         Brodo Naggins
84        Bilal Powell     NYJ   RB     7         Brodo Naggins
0         Dak Prescott     Dal   QB     6         Brodo Naggins
94       Justin Tucker     Bal    K     4         Brodo Naggins
24      DeSean Jackson      TB   WR     2         Brodo Naggins
120        James White      NE   RB     2         Brodo Naggins
108  Ricky Seals-Jones     Ari   TE     